In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np
import pandas as pd

In [10]:
STORM_NUMBER = 5
TITLE = "14-15 January 2022"
SUBTITLE = "8 AM 15 January 2022"
sts = utc(2022, 1, 15, 6)
ets = utc(2022, 1, 30, 20, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 60, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [11]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2021-2022 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df2)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [20]:
cull = [0, 'DSXI4', 'SXGI4', 156, 49, 50]
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["val"] < 1].index, USEME] = False
hardcode = [

]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=6, upper=8)
res = mp.postprocess(filename='220115.png')
mp.close()

     state  wfo   val      lon      lat                             geo  \
1       IA  DMX  12.9 -93.7200  41.7400  POINT (520636.294 -342538.038)   
2       IA  DMX  12.6 -93.6900  42.0300  POINT (520710.510 -310179.590)   
3       IA  DMX  12.0 -93.6300  42.0400  POINT (525565.521 -308687.684)   
4       IA  DMX  11.5 -92.3900  41.0500  POINT (637398.591 -409674.365)   
5       IA  DMX  11.5 -92.6400  41.2900  POINT (614229.587 -384979.166)   
...    ...  ...   ...      ...      ...                             ...   
1658    WI  GRB   0.0 -87.7356  45.0281    POINT (959231.190 76006.192)   
1660    WI  GRB   0.0 -88.6183  44.7631    POINT (894912.758 36652.353)   
1662    WI  GRB   0.0 -89.4380  45.9147   POINT (814196.107 155137.197)   
1663    WI  GRB   0.0 -89.5855  44.5113    POINT (822990.317 -1418.857)   
1664    WI  GRB   0.0 -90.0899  44.8175    POINT (779244.616 27440.362)   

      used_for_analysis  nwsli  plotme source  xcell  ycell  
1                  True      1    Tru